In [1]:
import pandas as pd
import numpy as np

In [8]:
file = "../model_saves/delay_ladder_20240624_214924/out.csv"
pd.read_csv(file).columns

Index(['step', 'episode', 'mean_length', 'std_length', 'mean_attacker_reward',
       'std_attacker_reward', 'mean_attacker_valid', 'std_attacker_valid',
       'mean_attacker_invalid', 'std_attacker_invalid', 'mean_defender_reward',
       'std_defender_reward', 'mean_defender_valid', 'std_defender_valid',
       'mean_defender_invalid', 'std_defender_invalid',
       'mean_network_avilability', 'std_network_avilability',
       'min_network_avilability', 'mean_network_infected',
       'std_network_infected'],
      dtype='object')

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os

matplotlib.use('Agg')

def plot_data(evaluate=False, lag=0, attacker=True):
    """
    Plots differences in valid actions for attacker and defender over training cycles,
    and network metrics (availability and infection).

    Parameters:
    evaluate (bool): If True, plots from 'out_eval.csv'. Otherwise, plots from 'out.csv'.
    """
    # File selection and configuration
    file_name = 'out_eval.csv' if evaluate else 'out.csv'
    max_step = 9 if evaluate else 19
    highlight_every = 2 if evaluate else 12
    highlight_span = 1 if evaluate else 6
    max_episodes = 20

    # Directory where data files are stored 
    data_dir = "../model_saves/delay_ladder_20240624_214924"
    file_path = f'{data_dir}/{file_name}'

    # Read the data from the CSV file
    data = pd.read_csv(file_path)
    data = data.iloc[:max_step * max_episodes]  # Include only up to max_step
    print(data)

    # Define the columns for attacker and defender valid actions
    columns_to_plot = {
        "Attacker Valid Actions": "mean_attacker_valid",
        "Defender Valid Actions": "mean_defender_valid"
    }

    # Create plots
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))

    # Assign subplots to attackers on the left and defenders on the right
    subplot_mapping = {
        "Attacker Valid Actions": axs[0, 0],
        "Defender Valid Actions": axs[0, 1],
    }

    # Initialize previous values for computing differences
    previous_values = {key: 0 for key in columns_to_plot.keys()}

    for title, col in columns_to_plot.items():
        ax = subplot_mapping[title]
        differences = []

        # Compute differences
        for i in range(len(data)):
            if i % highlight_every == 0 and i != 0:
                previous_values[title] = data[col].iloc[i-1]  # Update previous value at switch point
            differences.append((data[col].iloc[i] - previous_values[title]) / previous_values[title])

        # Plot differences
        ax.plot(data.index, differences, label=f'{col} (Difference)')
        ax.set_title(f'{title} Differences')
        ax.set_xlabel('Step')
        ax.set_ylabel('Difference Value')
        ax.legend()

        # Highlight sections based on training type
        for j in range(0, len(data.index), highlight_every):
            start_highlight = j
            end_highlight = min(j + highlight_span, len(data.index) - 1)
            ax.axvspan(start_highlight, end_highlight, color='blue', alpha=0.1)  # Attacker training
            if j + highlight_span < len(data.index):
                ax.axvspan(end_highlight, min(j + highlight_every, len(data.index) - 1), color='red', alpha=0.1)  # Defender training

    # Plot network metrics
    ax_network = axs[1, 0]
    ax_network.plot(data.index, data['mean_network_avilability'], label='Mean Network Availability', color='blue')
    ax_network.fill_between(data.index, 
                            data['mean_network_avilability'] - data['std_network_avilability'], 
                            data['mean_network_avilability'] + data['std_network_avilability'], 
                            color='blue', alpha=0.2)

    ax_network.plot(data.index, data['mean_network_infected'], label='Mean Network Infected', color='green')
    ax_network.fill_between(data.index, 
                            data['mean_network_infected'] - data['std_network_infected'], 
                            data['mean_network_infected'] + data['std_network_infected'], 
                            color='green', alpha=0.2)

    ax_network.plot(data.index, data['min_network_avilability'], label='Min Network Availability', color='red')

    ax_network.set_title('Network Metrics')
    ax_network.set_xlabel('Step')
    ax_network.set_ylabel('Value')
    ax_network.legend()

    # Adjust layout and save plot
    plt.tight_layout()
    os.makedirs('plots', exist_ok=True)  # Ensure the plots directory exists
    plt.savefig(f'plots/{data_dir.split("/")[1]}_{"eval" if evaluate else "train"}_out_diff.png')

# Example of calling the function
plot_data(evaluate=False, lag=0, attacker=True)  # For training data


     step  episode  mean_length  std_length  mean_attacker_reward  \
0       0        0       2000.0         0.0               -1692.8   
1       1        0       2000.0         0.0                   2.4   
2       2        0       2000.0         0.0                   4.2   
3       3        0       2000.0         0.0                   0.0   
4       4        0       2000.0         0.0                   6.0   
..    ...      ...          ...         ...                   ...   
115     1       19       2000.0         0.0                   2.4   
116     2       19       2000.0         0.0                   9.0   
117     3       19       2000.0         0.0                  12.0   
118     4       19       2000.0         0.0                   5.4   
119     5       19       2000.0         0.0                   9.6   

     std_attacker_reward  mean_attacker_valid  std_attacker_valid  \
0              53.495420                224.2           18.701872   
1               2.939388         

/tmp/ipykernel_8644/2023883958.py:58: RuntimeWarning: divide by zero encountered in double_scalars
  differences.append((data[col].iloc[i] - previous_values[title]) / previous_values[title])
/tmp/ipykernel_8644/2023883958.py:58: RuntimeWarning: divide by zero encountered in double_scalars
  differences.append((data[col].iloc[i] - previous_values[title]) / previous_values[title])
